In [2]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import models

In [3]:
num_classes = 4  # You have four classes: 'green,' 'sandy,' 'marshy,' and 'rocky'
class_names = ['green', 'sandy', 'marshy', 'rocky']

In [4]:
# Initialize the ResNet-18 model
resnet = models.resnet18(pretrained=True)
# Modify the final fully connected layer
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

c:\Users\Shree\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Shree\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [6]:
dataset_path = 'E:/SIH_2023/TERRAIN'
train_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)


In [7]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

In [8]:
if __name__ == '__main__':
    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)
     # Training loop
    num_epochs = 3 # You can adjust the number of training epochs
    for epoch in range(num_epochs):
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = resnet(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch [{epoch + 1}/{num_epochs}] Loss: {loss.item()}")

    print("Training finished.")

    # Save the trained model
    torch.save(resnet.state_dict(), 'resnet_model.pth')

    # Load the trained model for prediction
    resnet.load_state_dict(torch.load('resnet_model.pth'))
    resnet.eval()

    # Define a function for prediction
    def predict_image(image_path):
        image = Image.open(image_path)
        image_tensor = transform(image).unsqueeze(0)  # Add a batch dimension
        with torch.no_grad():
            outputs = resnet(image_tensor)
            _, predicted = torch.max(outputs, 1)
            class_index = predicted.item()
            class_name = class_names[class_index]  # Map class index to class name
        return class_name

    # Address of the image to be predicted
    prediction_image_path = 'E:/grass1.jpg'

    # Perform prediction
    predicted_class = predict_image(prediction_image_path)
    print(f"Predicted class for : {predicted_class}")

    # Evaluation
    correct = 0
    total = 0
    with torch.no_grad():
        for data in train_loader:
            images, labels = data
            outputs = resnet(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy on the training dataset: {100 * correct / total}%")

Epoch [1/3] Loss: 0.5493406057357788
Epoch [2/3] Loss: 0.37252646684646606
Epoch [3/3] Loss: 0.45089736580848694
Training finished.
Predicted class for : sandy
Accuracy on the training dataset: 82.33173076923077%
